In [3]:
import os

In [4]:
%pwd

'c:\\Users\\SniPer\\Desktop\\TOMATO DISEASE PREDICTION\\TomatoCare-AI-Project\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\SniPer\\Desktop\\TOMATO DISEASE PREDICTION\\TomatoCare-AI-Project'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig: 
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmented: bool
    params_image_size: list

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        validation = self.config.validation
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "TOMATO PROJECT DATASET", "train")
        create_directories([
            Path(training.root_dir)
        ])
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir, "TOMATO PROJECT DATASET", "valid")
        create_directories([
            Path(validation.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            validation_data=Path(validation_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmented=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [10]:
import os 
import urllib.request as request
from zipfile import ZipFile
# import tensorflow as tf
import time 

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            
        )

        dataflow_kwargs = dict(
            target_size=tuple(self.config.params_image_size[:-1]),
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            shuffle=False,
            class_mode="categorical",
            **dataflow_kwargs
        )

        if self.config.params_is_augmented:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            shuffle=True,
            class_mode="categorical",
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)


    
    ## Training function
    def train(self):
        if not tf.executing_eagerly():
            tf.config.run_functions_eagerly(True)

        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )


        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps
            
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e 

[2025-03-09 15:10:52,041: INFO: common: yaml file: config\config.yaml loaded successifully]
[2025-03-09 15:10:52,052: INFO: common: yaml file: params.yaml loaded successifully]
[2025-03-09 15:10:52,055: INFO: common: Created directory at: artifacts]
[2025-03-09 15:10:52,058: INFO: common: Created directory at: artifacts\training]
[2025-03-09 15:10:52,060: INFO: common: Created directory at: artifacts\validation]
[2025-03-09 15:10:55,965: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 600 images belonging to 10 classes.
Found 6000 images belonging to 10 classes.


c:\Users\SniPer\.conda\envs\Tomato\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5


c:\Users\SniPer\.conda\envs\Tomato\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


375/375 ━━━━━━━━━━━━━━━━━━━━ 4261s 11s/step - accuracy: 0.4861 - loss: 6.6859 - val_accuracy: 0.7703 - val_loss: 2.0283
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 4267s 11s/step - accuracy: 0.7324 - loss: 2.6170 - val_accuracy: 0.7179 - val_loss: 3.8474
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 4260s 11s/step - accuracy: 0.7691 - loss: 2.7859 - val_accuracy: 0.8091 - val_loss: 2.3711
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 4267s 11s/step - accuracy: 0.8202 - loss: 2.0852 - val_accuracy: 0.8784 - val_loss: 1.3718
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 4065s 11s/step - accuracy: 0.8158 - loss: 2.4108 - val_accuracy: 0.8057 - val_loss: 2.6843
[2025-03-09 21:02:58,127: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
